In [ ]:
import os
PROJ_DIR = os.path.expanduser("~/projects/bsp")

import pandas as pd
from matplotlib import pyplot as plt
plt.style.use("bmh")
COLORS = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
METHOD_ABBR_DICT = {
    "bsp": ("BSP", COLORS[0]),
    "ogn": ("OGN", COLORS[1]), 
    "in": ("IN", COLORS[2]),
    "mlpforce": ("MLP (Force)", COLORS[3]), 
    "mlpdynamics": ("MLP (Position)", COLORS[4]),
    "zero": (r"$F_0$", "gray"),
    
    "g11": (r"$G_{11}$", COLORS[0]),
    "g10": (r"$G_{10}$", COLORS[1]),
    "g01": (r"$G_{01}$", COLORS[2]),
    "g00": (r"$G_{00}$", COLORS[3]),
}

def make_efficiency_plot(dataset, methods, figsize=(4, 3), savefig=False, suffix=None, logscale=False):
    fig, ax = plt.subplots(figsize=figsize, tight_layout=True)

    for (idx, method) in enumerate(methods):
        
        method_display, color = METHOD_ABBR_DICT[method]
        
        fn = f"{PROJ_DIR}/paper/raw/efficiency-{dataset}-{method}.csv"
        df = pd.read_csv(fn, names=["nRMSE", "#Scenes", "Seed", "Seed (Data)"])
        nRMSE = df.groupby("#Scenes")["nRMSE"]
        
        if method == r"$F_0$":
            y = [nRMSE[0]]
            ax.hlines(y, 1, 10, color=color, label=method_display, linestyles="dashed")
        else:
            x = range(1, 10 + 1)
            y = nRMSE.median()
            yl = nRMSE.quantile(0.25)
            yh = nRMSE.quantile(0.75)
            ax.plot(x, y, "-x", color=color, label=method_display, alpha=0.8)
            plt.fill_between(x, yl, yh, color=color, alpha=0.2)

    ax.set_xlabel("No. of scenes in training")
    ax.set_ylabel("nRMSE")
    ax.legend(ncol=2)
    if logscale: ax.set_yscale("log")
    if savefig: fig.savefig(f"{PROJ_DIR}/paper/figs/efficiency-{dataset + '-' + suffix if suffix else dataset}.pdf", bbox_inches="tight")
    plt.close(fig)
    return fig

## Figure 4

In [ ]:
# Plots with selected methods
# This is needed as some methods are too bad and make it hard to see the difference
make_efficiency_plot("nbody", ["bsp", "ogn", "in", "mlpforce", "zero"], savefig=True)

In [ ]:
make_efficiency_plot("bounce", ["bsp", "ogn", "mlpforce", "zero"], savefig=True)

In [ ]:
make_efficiency_plot("mat", ["bsp", "ogn", "mlpforce", "zero"], savefig=True)

In [ ]:
# Also save version with all methods
for dataset in ["nbody", "bounce", "mat"]:
    fig = make_efficiency_plot(dataset, ["bsp", "ogn", "in", "mlpforce", "mlpdynamics", "zero"], savefig=True, suffix="full")

## Figure 5

In [ ]:
make_efficiency_plot("nbody", ["g11", "g01", "g00"], savefig=True, figsize=(4, 2), suffix="ablation")

In [ ]:
make_efficiency_plot("bounce", ["g11", "g01", "g00"], savefig=True, figsize=(4, 2), suffix="ablation", logscale=True)

In [ ]:
make_efficiency_plot("mat", ["g11", "g01", "g00"], savefig=True, figsize=(4, 2), suffix="ablation")